In [1]:
## 读取数据
import pandas as pd
features_filtered_direct=pd.read_pickle('extracted_train_features.pkl')
features_filtered_direct
y = pd.read_pickle('label.pkl')
y

d:\learn\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\learn\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
d:\learn\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


0       0
1       0
10      0
100     1
1000    0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: label, Length: 28388, dtype: int64

In [2]:
## 导入LightGBM模型
from lightgbm.sklearn import LGBMClassifier
import re
## 定义 LightGBM 模型 
clf = LGBMClassifier(is_unbalance=True)
# 在训练集上训练LightGBM模型
features_filtered_direct.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in features_filtered_direct.columns] 

clf.fit(features_filtered_direct, y)

# 成绩0.8308！！
testdata = pd.read_pickle('extracted_test_features.pkl')
testdata.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in testdata.columns] 
predict = clf.predict_proba(testdata)
print(predict)

##id 要对上！！！！！
result_csv = pd.read_csv('./result_automl_mljar.csv')
#result_csv['file_name']=testdata.index+'.pkl'
result_csv['score']= predict[:,-1]
result_csv.to_csv('result_lgm_(is unbalance).csv',index=None)

LGBMClassifier(is_unbalance=True)

[[9.99838095e-01 1.61905050e-04]
 [9.99920294e-01 7.97055001e-05]
 [9.99931815e-01 6.81845573e-05]
 ...
 [9.99949542e-01 5.04580841e-05]
 [9.99944661e-01 5.53387836e-05]
 [9.99950711e-01 4.92893447e-05]]


# 数据增强扩充相对提升一点

In [2]:
from imblearn.over_sampling import RandomOverSampler  
from collections import Counter
smt = RandomOverSampler(random_state=42)
X_resampled, y_resampled = smt.fit_resample(features_filtered_direct, y)
print(sorted(Counter(y_resampled).items()))

X_resampled.shape, y_resampled.shape

[(0, 23734), (1, 23734)]


((47468, 3291), (47468,))

## 特征选择


In [2]:
## 导入LightGBM模型
from lightgbm.sklearn import LGBMClassifier
import re
## 定义 LightGBM 模型 
clf = LGBMClassifier()
# 在训练集上训练LightGBM模型
X_resampled.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in X_resampled.columns] 

clf.fit(X_resampled, y_resampled)

LGBMClassifier()

In [4]:
# 成绩0.86！！
testdata = pd.read_pickle('extracted_test_features.pkl')
testdata.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in testdata.columns] 
predict = clf.predict_proba(testdata)
print(predict)

##id 要对上！！！！！
result_csv = pd.read_csv('./result_automl_mljar.csv')
#result_csv['file_name']=testdata.index+'.pkl'
result_csv['score']= predict[:,-1]
result_csv.to_csv('result_lgm.csv',index=None)

[[9.99737143e-01 2.62857463e-04]
 [9.99781330e-01 2.18669896e-04]
 [9.99924692e-01 7.53076863e-05]
 ...
 [9.99928151e-01 7.18486104e-05]
 [9.99916917e-01 8.30828311e-05]
 [9.99874711e-01 1.25288537e-04]]


In [5]:
from sklearn.feature_selection import SelectFromModel
model_select = SelectFromModel(clf, prefit=True)
X_new_train = model_select.transform(X_resampled)
X_new_train.shape

d:\learn\Anaconda3\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


(47468, 800)

In [7]:
X_new_train

array([[ 3.24355386e+01,  3.22249678e+00,  1.62817301e+01, ...,
         2.55858096e-01,  1.69800330e-01,  1.47507836e-02],
       [ 8.10888466e+01,  8.05624195e+00,  4.07043251e+01, ...,
         1.72603272e-01,  3.21552155e-01,  2.86647718e-01],
       [ 7.41383740e+01,  7.36570693e+00,  3.72153830e+01, ...,
        -7.39531315e-02,  1.05104798e-01, -9.22206290e-02],
       ...,
       [ 6.25542531e+01,  6.21481522e+00,  3.14004794e+01, ...,
         7.41001131e-02,  3.17700764e-02,  8.75592645e-02],
       [ 6.25542531e+01,  6.21481522e+00,  3.14004794e+01, ...,
         1.60942951e-02,  1.91440658e-01,  1.33070547e-01],
       [ 6.25542531e+01,  6.21481522e+00,  3.14004794e+01, ...,
         3.70250230e-02,  9.77662747e-02,  1.85449985e-01]])

In [6]:
X_new_train_df= pd.DataFrame(X_new_train,
index=X_resampled.index,columns=X_resampled.columns[model_select.get_support()])
X_new_train_df

,mintempcwtcoefficientscoeff2w20widths251020,mintempcwtcoefficientscoeff6w2widths251020,mintempcwtcoefficientscoeff1w20widths251020,mintempcwtcoefficientscoeff0w2widths251020,mintempcwtcoefficientscoeff10w2widths251020,mintempcwtcoefficientscoeff9w5widths251020,mintempcwtcoefficientscoeff9w2widths251020,mintempcwtcoefficientscoeff14w5widths251020,mintempcwtcoefficientscoeff14w2widths251020,mintempcwtcoefficientscoeff13w5widths251020,...,currentenergyratiobychunksnumsegments10segmentfocus7,currentquantileq06,currentquantileq09,currentabsenergy,currentc3lag2,currenttimereversalasymmetrystatisticlag2,currentsumofreoccurringdatapoints,currentpartialautocorrelationlag2,currentpartialautocorrelationlag3,currentpartialautocorrelationlag4
0,32.435539,3.222497,16.281730,0.001532,0.003064,58.010140,0.017069,8.980331,0.003064,14.333653,...,0.098094,-11.1,-11.1,31765.32,-1381.786607,6.983135e-01,-2840.9,0.255858,0.169800,0.014751
1,81.088847,8.056242,40.704325,0.003830,0.007661,145.025351,0.042671,22.450826,0.007661,35.834134,...,0.097505,-5.6,-5.6,8040.61,-176.004226,2.511111e-02,-1429.2,0.172603,0.321552,0.286648
2,74.138374,7.365707,37.215383,0.003502,0.007004,132.594606,0.039014,20.526470,0.007004,32.762636,...,0.094843,-11.1,-10.8,31734.79,-1381.516964,-1.443230e+00,-2839.0,-0.073953,0.105105,-0.092221
3,71.821550,7.135529,36.052402,0.003393,0.006785,128.451025,0.037795,19.885018,0.006785,31.738804,...,0.097705,-5.1,-5.1,6665.79,-132.868357,5.639228e-17,-1301.3,0.084588,0.194963,0.057652
4,69.504726,6.905350,34.889422,0.003283,0.006566,124.307443,0.036575,19.243565,0.006566,30.714972,...,0.095037,-4.1,-4.1,4457.10,-72.601885,7.116667e-02,-1068.0,0.231088,0.094958,0.075919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47463,74.138374,7.365707,37.215383,0.003502,0.007004,132.594606,0.039014,20.526470,0.007004,32.762636,...,0.097693,-4.1,-4.1,4301.76,-68.880976,-5.639228e-17,-1045.5,-0.003999,-0.004046,-0.004094
47464,62.554253,6.214815,31.400479,0.002955,0.005910,111.876699,0.032918,17.319209,0.005910,27.643475,...,0.097686,-5.1,-5.1,6656.54,-132.589071,1.127846e-16,-1305.4,-0.008062,-0.008190,-0.008321
47465,74.138374,7.365707,37.215383,0.003502,0.007004,132.594606,0.039014,20.526470,0.007004,32.762636,...,0.097675,-4.1,-4.1,4302.55,-68.900988,0.000000e+00,-1045.5,-0.003999,-0.004046,-0.004094
47466,62.554253,6.214815,31.400479,0.002955,0.005910,111.876699,0.032918,17.319209,0.005910,27.643475,...,0.099207,-5.3,-5.1,7078.62,-145.513500,-1.716825e-01,-1346.0,0.252294,0.271077,0.049388


In [ ]:
import numpy as np
testdata_new = np.load('testdata_new.npy')
extracted_test_features = pd.read_pickle('extracted_test_features.pkl')
testdata_new_df = pd.DataFrame(testdata_new, index = extracted_test_features.index,columns=X_new_train_df.columns)
testdata_new_df.to_pickle('testdata_new_df.pkl')
testdata_new_df

In [3]:
X_resampled.to_pickle('./X_resampled3.pkl')
y_resampled.to_pickle('./y_resampled3.pkl')


In [1]:
import pandas as pd
X_resampled=pd.read_pickle('./X_resampled3.pkl')
y_resampled=pd.read_pickle('./y_resampled3.pkl')

d:\learn\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\learn\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
d:\learn\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import re
X_resampled.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in X_resampled.columns] 

## FLAML

In [3]:
from flaml import AutoML


# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 20000,  # in seconds
    "metric": 'roc_auc',
    "task": 'classification',
    "log_file_name": "FLAML-2.log",
    "seed":2022
}

automl.fit(X_train=X_resampled, y_train=y_resampled,
           **automl_settings)
# Predict
print(automl.predict_proba(X_resampled))
# Print the best model
print(automl.model.estimator)

[flaml.automl: 10-10 23:21:23] {2600} INFO - task = classification
[flaml.automl: 10-10 23:21:23] {2602} INFO - Data split method: stratified
[flaml.automl: 10-10 23:21:23] {2605} INFO - Evaluation method: holdout
[flaml.automl: 10-10 23:21:26] {2727} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 10-10 23:21:26] {2869} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 10-10 23:21:26] {3174} INFO - iteration 0, current learner lgbm
[flaml.automl: 10-10 23:21:31] {3308} INFO - Estimated sufficient time budget=201623s. Estimated necessary time budget=4950s.
[flaml.automl: 10-10 23:21:31] {3360} INFO -  at 53.1s,	estimator lgbm's best error=0.0434,	best estimator lgbm's best error=0.0434
[flaml.automl: 10-10 23:21:31] {3174} INFO - iteration 1, current learner lgbm
[flaml.automl: 10-10 23:21:36] {3360} INFO -  at 58.0s,	estimator lgbm's best error=0.0233,	best estimator lgbm's best error=0.0233


[[9.99999391e-01 6.08815522e-07]
 [9.99989941e-01 1.00594576e-05]
 [9.99999720e-01 2.80224934e-07]
 ...
 [2.95558763e-06 9.99997044e-01]
 [1.59779621e-06 9.99998402e-01]
 [1.25155963e-05 9.99987484e-01]]
LGBMClassifier(colsample_bytree=0.4170609434684847,
               learning_rate=0.8526687043856349, max_bin=255,
               min_child_samples=11, n_estimators=165, num_leaves=91,
               reg_alpha=0.04028591453492402, reg_lambda=0.2423391921936627,
               verbose=-1)


In [4]:
# Best model and metric
'''retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 165, 'num_leaves': 91, 'min_child_samples': 11, 'learning_rate': 0.8526687043856349, 'log_max_bin': 8, 'colsample_bytree': 0.4170609434684847, 'reg_alpha': 0.04028591453492402, 'reg_lambda': 0.2423391921936627, 'FLAML_sample_size': 42721}
Best accuracy on validation data: 1
Training duration of best run: 39.59 s


In [ ]:
'''pickle and save the automl object'''
import pickle
with open('automl.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [5]:
# Predict
testdata= pd.read_pickle('extracted_test_features.pkl')
testdata.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in testdata.columns] 

predict = automl.predict_proba(testdata)
print(predict)

[[9.99998243e-01 1.75706492e-06]
 [9.99999762e-01 2.37593865e-07]
 [9.99986986e-01 1.30139883e-05]
 ...
 [9.99999280e-01 7.20398311e-07]
 [9.99999750e-01 2.50052593e-07]
 [9.99999884e-01 1.15907289e-07]]


In [7]:

##id 要对上！！！！！
result_csv = pd.read_csv('result_automl_mljar.csv')
#result_csv['file_name']=testdata.index+'.pkl'
result_csv['score']= predict[:,-1]
result_csv.to_csv('result_automl_flaml.csv',index=None)

## mljar

In [7]:
import numpy as np
from supervised import AutoML
y = pd.Series(y_resampled.values, index=y_resampled.index)
y
automl_mljar = AutoML(mode="Compete", features_selection=True,total_time_limit=100000,
results_path= './OUTPUT/best/mljar_compete_feature_800',random_state=42)
automl_mljar.fit(X_new_train_df, y)

Linear algorithm was disabled.
AutoML directory: ./OUTPUT/best/mljar_compete_feature_800
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.158087 trained in 11.97 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.159833 trained in 101.37 seconds
2_DecisionTree logloss 0.284206 trained in 72.7 se

In [5]:
import numpy as np
from supervised import AutoML
y = pd.Series(y_resampled.values, index=y_resampled.index)
y
automl_mljar = AutoML(mode="Compete", features_selection=True,total_time_limit=10000,
results_path= './OUTPUT/best/mljar_compete4',random_state=2022)
automl_mljar.fit(X_resampled, y)

Linear algorithm was disabled.
AutoML directory: ./OUTPUT/best/mljar_compete4
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.159555 trained in 42.65 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 5-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.168049 trained in 182.91 seconds
2_DecisionTree logloss 0.214584 trained in 166.44 seconds
3_De

AutoML(features_selection=True, mode='Compete', random_state=2022,
       results_path='./OUTPUT/best/mljar_compete4', total_time_limit=10000)

In [ ]:
import numpy as np
from supervised import AutoML

automl_mljar = AutoML(mode="Compete", features_selection=True,total_time_limit=10000,
results_path= './OUTPUT/best/mljar_compete2',random_state=2022)
automl_mljar.fit(X_resampled, y)

In [6]:
import pandas as pd
testdata = pd.read_pickle('extracted_test_features.pkl')
testdata

,"min_temp__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_1__w_20__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_10__w_2__widths_(2, 5, 10, 20)","min_temp__cwt_coefficients__coeff_9__w_5__widths_(2, 5, 10, 20)",...,current__partial_autocorrelation__lag_1,current__partial_autocorrelation__lag_9,current__partial_autocorrelation__lag_6,current__partial_autocorrelation__lag_5,current__partial_autocorrelation__lag_2,current__partial_autocorrelation__lag_3,current__partial_autocorrelation__lag_8,current__partial_autocorrelation__lag_7,current__partial_autocorrelation__lag_4,current__longest_strike_below_mean
0,60.237429,1.331507,5.984637,30.237499,0.011249,0.007900,0.005435,0.002845,0.005691,107.733118,...,0.778826,-0.109876,-0.063959,0.074779,0.356574,0.182475,0.142746,-0.051169,-0.005029,71.0
1,20.851418,0.460906,2.071605,10.466826,0.003894,0.002735,0.001881,0.000985,0.001970,37.292233,...,0.923448,0.042001,-0.034045,-0.032112,0.137298,-0.001054,0.153169,-0.035414,-0.026645,37.0
10,60.237429,1.331507,5.984637,30.237499,0.011249,0.007900,0.005435,0.002845,0.005691,107.733118,...,0.828580,0.070565,0.127968,0.092518,0.238113,0.101794,0.085683,0.093250,-0.111057,27.0
10001,55.603781,1.229083,5.524280,27.911537,0.010384,0.007292,0.005017,0.002627,0.005253,99.445955,...,-0.015998,-0.019518,-0.018072,-0.017626,-0.016383,-0.016782,-0.019016,-0.018535,-0.017196,1.0
10002,78.772022,1.741201,7.826064,39.541344,0.014710,0.010331,0.007107,0.003721,0.007442,140.881769,...,-0.028333,-0.038988,-0.034298,-0.032945,-0.029382,-0.030495,-0.037306,-0.035747,-0.031681,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,106.573913,2.355742,10.588204,53.497113,0.019902,0.013977,0.009616,0.005034,0.010068,190.604746,...,0.679132,0.008755,0.028377,0.152659,0.228818,0.212332,0.074055,0.239131,0.179926,116.0
9995,31.915087,1.274917,4.412830,15.914968,-0.207792,0.055820,-1.849390,-1.213677,0.561016,60.410570,...,0.941508,-0.041501,-0.106493,-0.264966,0.356769,0.601220,0.115021,-0.334717,0.138343,56.0
9996,60.237429,1.331507,5.984637,30.237499,0.011249,0.007900,0.005435,0.002845,0.005691,107.733118,...,0.963371,-0.047714,-0.031041,-0.030952,-0.140295,-0.047587,-0.024204,0.277707,-0.026313,100.0
9998,16.217769,0.358483,1.611248,8.140865,0.003029,0.002127,0.001463,0.000766,0.001532,29.005070,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [ ]:
testdata = pd.read_pickle('extracted_test_features.pkl')
testdata
predict = automl_mljar.predict_proba(testdata)
print(predict)

##id 要对上！！！！！
result_csv = pd.read_csv('./result_automl_784_params_1.csv')
#result_csv['file_name']=testdata.index+'.pkl'
result_csv['score']= predict[:,-1]
result_csv.to_csv('result_automl_mljar.csv',index=None)

In [ ]:

predict = automl_mljar.predict_proba(testdata_new_df)
print(predict)

##id 要对上！！！！！
result_csv = pd.read_csv('./result/result_lgm.csv')
#result_csv['file_name']=testdata.index+'.pkl'
result_csv['score']= predict[:,-1]
result_csv.to_csv('result_automl_mljar_800features.csv',index=None)

[[9.99995484e-01 4.50971371e-06]
 [9.99997829e-01 2.17690474e-06]
 [9.99996237e-01 3.76410902e-06]
 ...
 [9.99997112e-01 2.88970103e-06]
 [9.99997932e-01 2.07010600e-06]
 [9.99998193e-01 1.80683468e-06]]


## evalml 效果一般，废弃

In [14]:
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types

d:\learn\Anaconda3\lib\site-packages\woodwork\__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
Featuretools may not support Python 3.7 in next non-bugfix release.


In [15]:
y = pd.Series(y_resampled.values, index=y_resampled.index)
y

0        0
1        0
2        0
3        1
4        0
        ..
47463    1
47464    1
47465    1
47466    1
47467    1
Length: 47468, dtype: int64

In [16]:
automl_evalml = AutoMLSearch(
    X_train=X_new_train_df,
    y_train=y,
    problem_type="binary",
    ensembling=True,
    max_batches=500,
    
    patience=6,
    random_seed=2022,
)

In [17]:
automl_evalml.search()

			Elastic Net Classifier w/ Label Encoder + Imputer + Standard Scaler fold 0: Encountered an error.
			Elastic Net Classifier w/ Label Encoder + Imputer + Standard Scaler fold 0: All scores will be replaced with nan.
			Fold 0: Exception during automl search: 'TableSchema' object has no attribute '_get_subset_schema'
			Fold 0: Parameters:
	{'Label Encoder': {'positive_label': None}, 'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Elastic Net Classifier': {'penalty': 'elasticnet', 'C': 1.0, 'l1_ratio': 0.15, 'n_jobs': -1, 'multi_class': 'auto', 'solver': 'saga'}}
			Fold 0: Traceback:
  File "C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\evalml\automl\engine\engine_base.py", line 224, in train_and_score_pipeline
    pipeline, X_train, y_train, automl_config, schema=False

  File "C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\evalml\

In [ ]:
automl_evalml.rankings()

## H2O 效果一般，废弃

In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

In [ ]:
import pandas as pd
X_resampled=pd.read_pickle('./X_resampled.pkl')
y_resampled=pd.read_pickle('./y_resampled.pkl')

In [ ]:
X_resampled.to_csv('X_resampled.csv',index=False,encoding='utf-8')
y_resampled.to_csv('y_resampled.csv',index=False,encoding='utf-8')

In [ ]:
X_resampled

In [ ]:
## 完整数据集
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_full_train, X_full_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=.2)

## 导入LightGBM模型
from lightgbm.sklearn import LGBMClassifier
import re
## 定义 LightGBM 模型 
X_full_train.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in X_full_train.columns] 
X_full_test.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in X_full_test.columns] 

clf = LGBMClassifier()
# 在训练集上训练LightGBM模型
clf.fit(X_full_train, y_train)
train_predict = clf.predict(X_full_train)
test_predict = clf.predict(X_full_test)
from sklearn import metrics

## 利用accuracy（准确度）【预测正确的样本数目占总预测样本数目的比例】评估模型效果
print('The train-set accuracy is:',metrics.f1_score(y_train,train_predict))
print('The test-set accuracy is:',metrics.f1_score(y_test,test_predict))
## 查看混淆矩阵 (预测值和真实值的各类情况统计矩阵)
confusion_matrix_result_train = metrics.confusion_matrix(train_predict,y_train)
confusion_matrix_result_test = metrics.confusion_matrix(test_predict,y_test)
print('The trainset confusion matrix result:\n',confusion_matrix_result_train)
print('\n The testset confusion matrix result:\n',confusion_matrix_result_test)
print('\n The trainset classification_report:\n',classification_report(train_predict,y_train))
print('\n The testset classification_report:\n',classification_report(test_predict,y_test))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

model_select = SelectFromModel(clf, prefit=True,max_features=2000)
X_new_train = model_select.transform(X_full_train)
X_new_train.shape

In [ ]:
X_new_train

In [ ]:
X_new_train_df= pd.DataFrame(X_new_train,
index=X_full_train.index,columns=X_full_train.columns[model_select.get_support()])
X_new_train_df

In [ ]:
import numpy as np

# Generate a random dataset with 10 rows 4 columns.
# Label the columns A, B, C, and D.
#train = h2o.H2OFrame.from_python(np.random.randn(10,4).tolist(), column_names=list('ABCD'))
train = h2o.H2OFrame.from_python(X_resampled)
train.describe

In [ ]:
train

In [ ]:
y_resampled

In [ ]:
y_train=np.array(y_train)
label = h2o.H2OFrame.from_python(y_train,column_names=list('y'))
label

In [ ]:
train = train.cbind(label)
train.describe

In [ ]:
# Change the column type to a factor: f分类问题！
train['y'] = train['y'].asfactor()

In [ ]:

h2o.export_file(train, path = "/content/drive/MyDrive/baidu_dataset/trainHEX.csv", force = True)

In [ ]:
# begin！！！！！！！！！！
import h2o
h2o.init()
train = h2o.import_file("trainHEX.csv")

In [ ]:
# Identify predictors and response
x = train.columns
y = "y"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [ ]:
x

In [ ]:

from h2o.automl import H2OAutoML
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=10, seed=2022)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

In [ ]:
GBM_1_AutoML = h2o.get_model('StackedEnsemble_BestOfFamily_1_AutoML_3_20221004_93715')
GBM_1_AutoML

In [ ]:
DRF_1_AutoML = h2o.get_model('DRF_1_AutoML_3_20221004_93715')

In [ ]:
# Get the best model using the metric
m = aml.leader
# this is equivalent to
m = aml.get_best_model()
m

In [ ]:
# save the model
model_path = h2o.save_model(model=m, path="./OUTPUT/best", force=True)
print( model_path)

In [ ]:
## ceshiji

In [ ]:
# 测试集
import re
import pandas as pd
#testdata= np.load('/content/drive/MyDrive/baidu_dataset/testdata_new.npy')
testdata= pd.read_pickle('extracted_test_features.pkl')
testdata2 = testdata
testdata2.columns = [re.sub('[^A-Za-z0-9]+', '', col) for col in testdata2.columns] 
testdata_new = testdata2[list(x)]
print(testdata_new.shape)
#np.save("testdata_new.npy",testdata_new)

In [ ]:
testdata_new = testdata2[list(x)]

In [ ]:
testdata_new 


In [ ]:
testdata_new_df.iloc[-1,:]

In [ ]:
testdata_hex = h2o.H2OFrame.from_python(testdata_new)
testdata_hex.describe
h2o.export_file(testdata_hex, path = "./testdata_hex.csv", force = True)

In [ ]:
preds = GBM_1_AutoML.predict(testdata_hex)
preds
# or:
#preds = aml.leader.predict(test)

In [ ]:
preds = DRF_1_AutoML.predict(testdata_hex)
preds

In [ ]:
preds.shape

In [ ]:
testdata2.index

In [ ]:
 preds[:,-1]

In [ ]:
pd.DataFrame(preds[:,-1],columns=['score'])

In [ ]:

h2o.export_file(preds[:,-1], path = "./result_automl_784_params_pre0.csv", force = True)

In [ ]:
result_csv = pd.read_csv('result_automl_784_params_pre0.csv')
result_csv

In [ ]:


##id 要对上！！！！！
result_csv = pd.read_csv('result_automl_784_params_pre0.csv')
result_csv['file_name']=testdata2.index+'.pkl'
result_csv['score']= result_csv['p1']
result_csv.to_csv('result_automl_784_params_1.csv',index=None)


## 结果融合

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import csv
import pandas as pd
def all_csv(file_PATH = r'./result',save_file="融合.csv"):
    df_list = [] #创建新列表用来存储提取出来的列表
    tqdm=os.listdir(file_PATH)#文件夹中的文件列表
    for i in range(0,len(tqdm)):#逐次遍历文件夹下的文件
        files_path = os.path.join(file_PATH,tqdm[i])#对应文件夹下的某份文件
        df = pd.read_csv(files_path)##读取CSV文件数据
        #这里实现提取第一份数据的x与y轴数据，其他文件只提取y轴数据
        if i ==0:
            data = df.iloc[:,:]#选取文件中某行某列数据
        else:
            data['score'] = (data['score']+data['score'])/(i+1)
        #df_list.append(data)#将选取的数据添加到列表
    #df2 = pd.concat(df_list,axis=1)#将列表数据按列合并，axis=1表示按列整合
    save_path=os.path.join(file_PATH,save_file)#定义保存存放整合后数据的路径
    data.to_csv(save_path,index=False)#将整合好的数据输入到新建的csv文件中



if __name__ == "__main__":
    all_csv(file_PATH = r'./result',save_file="融合.csv")

In [ ]:


##id 要对上！！！！！
result_csv = pd.read_csv('result_automl_784_params_pre0.csv')
result_csv['file_name']=testdata2.index+'.pkl'
result_csv['score']= result_csv['p1']
result_csv.to_csv('result_automl_784_params_1.csv',index=None)
